In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\x526378\Desktop\projetos\cebrap\coleta_cebrap
Caminho no path.


In [2]:
import os
import geopandas as gpd
import pandas as pd

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel
)
from utils.downloads import _prepare_cache

# Intersecção entre setores censitários (2022) e a área de 1km a partir de Parques Urbanos e Lineares (buff)

Pessoas de outras cidades também estão dentro das áreas de 1km de parques, porém optei por não considerá-las ao fazer a contagem.

Eu interpretei o indicador como <=1km, não como <1km

In [3]:
data_path = get_data_diretorio()

In [4]:
assets_path = os.path.join(
    data_path,
    'assets'
)

# Dependências

Este notebook depende dos arquivos resultantes dos notebooks '.buffed_parques_urb_line' e '../load_data/setores_censitario'

In [5]:
gdf_buffed_parq = gpd.read_parquet(
    os.path.join(
        assets_path,
        'parques_lineares_e_urbanos',
        'area_1km_parques_urbanos_lineares.parquet'        
    )
)

In [6]:
gdf_setores22= gpd.read_parquet(
    os.path.join(
        assets_path,
        'setores_2022.parquet'
    )
)

In [7]:
gdf_setores22.shape

(27301, 27)

# Overlay setores por buff

Antes de interseccionar, precisaremos ter a informação da área da geometria original, para poder fazer o cálculo da porcentagem da área:

In [8]:
gdf_setores22['area_setor']=gdf_setores22['geometry'].area

Usando o método overlay, fazemos a intersecção, que exluirá todos os setores que não estão na área do buffer dos parques. Os que estiverem parte dentro e parte fora, serão cortados seguindo a geometria do buff.

In [9]:
overlay_buff_setores= gpd.overlay(
    gdf_setores22,
    gdf_buffed_parq,
    how='intersection',
    keep_geom_type=True    
)

In [10]:
overlay_buff_setores.shape

(17752, 28)

Para apagarmos as geometrias desprezíveis eu usarei o mesmo padrão que usei para quadras viárias e outras malhas das Áreas Verdes Públicas: desprezar as geometrias menores que 1% da área total do menor setor

In [11]:
overlay_buff_setores['area_recort']=overlay_buff_setores['geometry'].area

In [12]:
gdf_setores22.loc[gdf_setores22['area_setor']==gdf_setores22['area_setor'].min()]

,CD_SETOR,SITUACAO,CD_SIT,CD_TIPO,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,...,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,total_pop,total_dom,geometry,area_setor
22732,355030878000084,Urbana,1,0,3550308,São Paulo,355030878,Sé,35503087800,None,...,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,283.0,130.0,"POLYGON ((333744.46 7394168.456, 333741.319 73...",246.391634


In [13]:
min_setor= float(gdf_setores22['area_setor'].min())
min_1pct = min_setor*0.01
irrelevants = overlay_buff_setores.loc[overlay_buff_setores['area_recort']<min_1pct]
irrelevants.shape

(16, 29)

In [14]:
overlay_buff_setores=overlay_buff_setores.loc[overlay_buff_setores['area_recort']>=min_1pct]
overlay_buff_setores.shape

(17736, 29)

In [15]:
setores_omitidos = gdf_setores22.loc[~gdf_setores22['CD_SETOR'].isin(overlay_buff_setores['CD_SETOR'])]
setores_omitidos
print(f'{len(setores_omitidos)} setores foram omitidos após a intersecção')
print(f'{len(gdf_setores22)-len(setores_omitidos)} setores interseccionam com o buff dos parques')

9590 setores foram omitidos após a intersecção
17711 setores interseccionam com o buff dos parques


# Calcular a população

Como não temos como saber precisamente onde as pessoas se encontram dentro do setore censitário, estabelecemos por padrão que faremos o cálculo como se elas estivessem distribuídas de forma homogênea no espaço. A partir disso, podemos assumir que a maior % de área do setor, também equivalerá a um recorte com a maior quantidade de pessoas do setor, apesar de isso não necessariamente acontecer na realidade.

Para calcular a porcentagem da área do recorte, vamos calcular a área do recorte a partir da geometria atual, e dividí-la pela área total do setor, anteriormente calculada:
$$ (\text{Porcentagem da área do setor})=\frac{(\text{área do recorte do setor censitário})}{(\text{área total do setor censitário})}$$ 

Agora que sabemos a porcentagem da área de cada recorte, podemos multiplicá-la (já que está em decimal) pelo total da população no setor e teremos a quantidade de pessoas nesta "fatia". Repare que, por estarmos calculando o número de pessoas, é necessário que seja um número inteiro (ou um número arredondado a 0 casa decimais):
$$ \text{Quantidade de pessoas no recorte do setor} = \text{quantidade total de pessoas em todo o setor}*\text{porcentagem da área do setor}$$

In [16]:
overlay_buff_setores['percent_area'] = overlay_buff_setores['area_recort']/overlay_buff_setores['area_setor']

In [17]:
overlay_buff_setores['pop_recort']=round(overlay_buff_setores['total_pop']*overlay_buff_setores['percent_area'], 0)

Como esse método de arredondamento não leva em conta os outros recortes para arredondar, é possível que o número de pessoas varie.

In [18]:
overlay_buff_setores.sample(2)

,CD_SETOR,SITUACAO,CD_SIT,CD_TIPO,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,...,NM_RGI,CD_CONCURB,NM_CONCURB,total_pop,total_dom,area_setor,geometry,area_recort,percent_area,pop_recort
8058,355030843000594,Urbana,1,0,3550308,São Paulo,355030843,Jardim Ângela,35503084300,None,...,São Paulo,3550308,São Paulo/SP,701.0,269.0,27749.541348,"POLYGON ((320643.003 7379467.447, 320628.699 7...",27749.541348,1.0,701.0
15935,355030887000188,Urbana,1,0,3550308,São Paulo,355030887,Vila Jacuí,35503088700,None,...,São Paulo,3550308,São Paulo/SP,332.0,130.0,10404.365998,"POLYGON ((350792.708 7399009.781, 350855.391 7...",10404.365998,1.0,332.0


Para visualizar melhor, vamos procurar só os casos que o setor não está completo:

In [19]:
overlay_buff_setores.loc[overlay_buff_setores['percent_area']<=0.99].sample(4)

,CD_SETOR,SITUACAO,CD_SIT,CD_TIPO,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_SUBDIST,NM_SUBDIST,...,NM_RGI,CD_CONCURB,NM_CONCURB,total_pop,total_dom,area_setor,geometry,area_recort,percent_area,pop_recort
8118,355030843000700,Urbana,1,4,3550308,São Paulo,355030843,Jardim Ângela,35503084300,None,...,São Paulo,3550308,São Paulo/SP,19.0,6.0,40893.035938,"POLYGON ((320311.367 7379742.074, 320297.344 7...",5475.523508,0.133899,3.0
5480,355030831000177,Urbana,1,1,3550308,São Paulo,355030831,Guaianases,35503083100,None,...,São Paulo,3550308,São Paulo/SP,650.0,249.0,35367.015905,"POLYGON ((357949.676 7393494.47, 357944.39 739...",23596.439933,0.667188,434.0
14405,355030878000106,Urbana,1,0,3550308,São Paulo,355030878,Sé,35503087800,None,...,São Paulo,3550308,São Paulo/SP,150.0,91.0,23674.706970,"POLYGON ((333527.595 7395464.109, 333557.79 73...",2886.129501,0.121908,18.0
9452,355030851000074,Urbana,1,0,3550308,São Paulo,355030851,Mandaqui,35503085100,None,...,São Paulo,3550308,São Paulo/SP,292.0,138.0,26908.633389,"POLYGON ((331865.065 7401945.196, 331858.779 7...",20985.134977,0.779866,228.0


# Salvar gdf

In [20]:
save_parquet_excel(
    overlay_buff_setores,
    fname='overlay_setores22_buff_parques_urb_line',
    data_path = assets_path,
    data_subpath = 'parques_lineares_e_urbanos'
)